In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf

In [2]:
os.chdir(os.path.dirname(os.getcwd()))

In [3]:
os.getcwd()

'C:\\Users\\noahc\\VC_Repo'

In [4]:
PATH = os.path.join(os.path.dirname(os.getcwd()), "VC_Repo\\data\\raw\\cv_delta\\scales")
PATH2 = 'data\\processed'
os.getcwd()



'C:\\Users\\noahc\\VC_Repo'

In [5]:
def main(input_filepath, output_filepath):
    """ Runs data processing scripts to turn raw data from (../raw) into
        cleaned data ready to be analyzed (saved in ../processed).
    """

    train_dir = os.path.join(input_filepath, 'train')
    test_dir = os.path.join(input_filepath, 'test')

    BATCH_SIZE = 32
    IMG_SIZE1= 224 
    IMG_SIZE = (IMG_SIZE1, IMG_SIZE1)
    
    IMG_SHAPE = (IMG_SIZE1, IMG_SIZE1, 3)

    train_dataset = tf.keras.utils.image_dataset_from_directory(train_dir,
                                                            shuffle=True,
                                                            batch_size=BATCH_SIZE,
                                                            image_size=IMG_SIZE,
                                                            )
    test_dataset = tf.keras.utils.image_dataset_from_directory(test_dir,
                                                                 shuffle=True,
                                                                 batch_size=BATCH_SIZE,
                                                                 image_size=IMG_SIZE,
                                                            )
    class_names = train_dataset.class_names
    val_batches = tf.data.experimental.cardinality(train_dataset)

    validation_dataset = train_dataset.take(val_batches // 5)

    train_dataset = train_dataset.skip(val_batches // 5)

    AUTOTUNE = tf.data.AUTOTUNE

    train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
    validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)
    test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)

    return (train_dataset, validation_dataset, test_dataset, class_names)


In [6]:
train_dataset, validation_dataset, test_dataset, class_names = main(input_filepath = PATH, output_filepath = PATH2)



Found 2956 files belonging to 2 classes.
Found 740 files belonging to 2 classes.


In [7]:
test_dataset
train_dataset

<PrefetchDataset shapes: ((None, 224, 224, 3), (None,)), types: (tf.float32, tf.int32)>

In [8]:
BATCH_SIZE = 32
IMG_SIZE1= 224 
IMG_SIZE = (IMG_SIZE1, IMG_SIZE1)
IMG_SHAPE = (IMG_SIZE1, IMG_SIZE1, 3)

preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input
rescale = tf.keras.layers.Rescaling(1./127.5, offset=-1)

data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip('horizontal'),
  tf.keras.layers.RandomRotation(0.2),
])

# Create the base model from the pre-trained model MobileNet V2
IMG_SHAPE = IMG_SIZE + (3,)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')
image_batch, label_batch = next(iter(train_dataset))
feature_batch = base_model(image_batch)
base_model.trainable = False

global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
print(feature_batch_average.shape)

prediction_layer = tf.keras.layers.Dense(1)
prediction_batch = prediction_layer(feature_batch_average)
print(prediction_batch.shape)

inputs = tf.keras.Input(IMG_SHAPE)
x = data_augmentation(inputs)
x = preprocess_input(x)
x = base_model(x, training=False)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)

base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

initial_epochs = 20

loss0, accuracy0 = model.evaluate(validation_dataset)

history = model.fit(train_dataset,
                    epochs=initial_epochs,
                    validation_data=validation_dataset)

base_model.trainable = True




(32, 1280)
(32, 1)
18/18 [==============================] - 3s 60ms/step - loss: 0.6792 - accuracy: 0.4132
Epoch 1/20
75/75 [==============================] - 9s 80ms/step - loss: 0.6916 - accuracy: 0.4975 - val_loss: 0.6824 - val_accuracy: 0.5868
Epoch 2/20
75/75 [==============================] - 6s 75ms/step - loss: 0.6840 - accuracy: 0.5147 - val_loss: 0.6792 - val_accuracy: 0.5903
Epoch 3/20
75/75 [==============================] - 6s 74ms/step - loss: 0.6840 - accuracy: 0.5126 - val_loss: 0.6722 - val_accuracy: 0.6042
Epoch 4/20
75/75 [==============================] - 6s 74ms/step - loss: 0.6746 - accuracy: 0.5441 - val_loss: 0.6747 - val_accuracy: 0.6076
Epoch 5/20
75/75 [==============================] - 6s 75ms/step - loss: 0.6860 - accuracy: 0.5231 - val_loss: 0.6932 - val_accuracy: 0.5868
Epoch 6/20
75/75 [==============================] - 6s 74ms/step - loss: 0.6892 - accuracy: 0.5181 - val_loss: 0.6886 - val_accuracy: 0.5833
Epoch 7/20
75/75 [=============================

KeyboardInterrupt: 

In [ ]:
# Fine-tune from this layer onwards
fine_tune_at = 100

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable = False

model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer = tf.keras.optimizers.RMSprop(learning_rate=base_learning_rate/15),
              metrics=['accuracy'])

fine_tune_epochs = 15
total_epochs =  initial_epochs + fine_tune_epochs

history_fine = model.fit(train_dataset,
                         epochs=total_epochs,
                         initial_epoch=history.epoch[-1],
                         validation_data=validation_dataset)



In [ ]:
y_pred = model.predict(test_dataset, batch_size=64, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)
loss, accuracy = model.evaluate(test_dataset)
print('Test accuracy :', accuracy)

# Retrieve a batch of images from the test set
image_batch, label_batch = test_dataset.as_numpy_iterator().next()
predictions = model.predict_on_batch(image_batch).flatten()

# Apply a sigmoid since our model returns logits
predictions = tf.nn.sigmoid(predictions)
predictions = tf.where(predictions < 0.5, 0, 1)

print('Predictions:\n', predictions.numpy())
print('Labels:\n', label_batch)
from sklearn.metrics import classification_report



print(classification_report(class_names, y_pred_bool))

plt.figure(figsize=(10, 10))
for i in range(9):
  ax = plt.subplot(3, 3, i + 1)
  plt.imshow(image_batch[i].astype("uint8"))
  plt.title(class_names[predictions[i]])
  plt.axis("off")